In [22]:
import pandas as pd
import tensorflow as tf
import glob
import os
import numpy as np
import sklearn.metrics as sk
import tensorflow_datasets as tfds
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

import seaborn as sns
import helpers
import dataset
import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
lie_trial_path = '/Users/frank/Downloads/dataSets/Trial-dataset/Clips/processed_lie/' #60 entries
truth_trial_path = '/Users/frank/Downloads/dataSets/Trial-dataset/Clips/processed_truth/' #61 entries

MU3D_path = '/Users/frank/Downloads/dataSets/MU3D-dataset/processed/' # 320 entries

In [3]:
X, Y = dataset.path_preprocessing(truth_trial_path, lie_trial_path)

In [8]:
TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

In [141]:
LSTM_NEURONS = 125
DROPOUT = 0.2
EPOCHS = 40
OPTIM = 'adam'
LOSS = 'binary_crossentropy' # cross entropy/ we are doing regression, not classification!

n_inputRows = xTrain.shape[0] 
n_timesteps = xTrain.shape[1]
n_features = xTrain.shape[2]

def LSTM_Model(neuro, drop, loss, optim, step, feat):
  model = Sequential()
  model.add(LSTM(neuro, return_sequences=False, batch_input_shape=(None, step, feat)))


  model.add(Dropout(drop))
  model.add(Dense(1, activation='relu')) #relu
  model.add(Dense(1, activation='softmax')) #softmax
  #model.add(Lambda(lambda x: tf.cast(tf.argmax(x), tf.float32)))
  #model.add(Dense(1, activation=activ))

  model.compile(loss=loss, optimizer=optim, metrics=['accuracy'])
  return model

MODEL = LSTM_Model(LSTM_NEURONS, DROPOUT, LOSS, OPTIM, n_timesteps, n_features)

MODEL.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=EPOCHS, verbose=1)
MODEL.summary()

Epoch 1/40


ValueError: in user code:

    File "/Users/frank/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/frank/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/frank/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/frank/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/frank/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "/Users/frank/opt/anaconda3/lib/python3.8/site-packages/keras/engine/compile_utils.py", line 265, in __call